In [1078]:
# Install eli5, you can use any other model Interpretability libraries
!pip install eli5 -q

In [42]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import eli5
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [43]:
train = pd.read_csv('/home/fjaadari/Downloads/Train (4).csv')
test = pd.read_csv('/home/fjaadari/Downloads/Test (4).csv')
ss = pd.read_csv('/home/fjaadari/Downloads/SampleSubmission (5).csv')


## Identifying and Dropping Columns with High Percentage of Missing Values

In [44]:
percent_missing = train.isnull().sum() * 100/ len(train)
missing_value_train = pd.DataFrame({'column_name': train.columns,
                                  'percent_missing': percent_missing})
columns_to_drop = list(percent_missing[percent_missing >= 40].index)
missing_value_train.sort_values(by="percent_missing",ascending=True)


,column_name,percent_missing
child_id,child_id,0.000000
pre_covid,pre_covid,0.000000
sef_ind,sef_ind,0.000000
language_assessment,language_assessment,0.000000
hle_ind,hle_ind,0.000000
...,...,...
pra_class_space_small,pra_class_space_small,99.895166
pri_founderother,pri_founderother,99.906814
pri_clinic_travelother,pri_clinic_travelother,99.941759
pri_locationother,pri_locationother,99.941759


In [45]:
len(columns_to_drop)

538

## return back relevant features

In [46]:
columns_to_drop.remove("child_attends")
columns_to_drop.remove("child_attendance")
columns_to_drop.remove("pra_salary")
columns_to_drop.remove("obs_area")
columns_to_drop.remove("obs_handwashing")
columns_to_drop.remove("obs_toilet")
columns_to_drop.remove("obs_building")
columns_to_drop.remove("obs_shared")
columns_to_drop.remove("obs_outdoor")
columns_to_drop.remove("obs_gate")
columns_to_drop.remove("obs_access")
columns_to_drop.remove("count_register_race_coloured")
columns_to_drop.remove("count_register_race_white")
columns_to_drop.remove("count_staff_qual_skills")
columns_to_drop.remove("urban")







In [47]:
train = train.drop(columns = columns_to_drop)
test = test.drop(columns = columns_to_drop )
train.shape, test.shape



((8585, 156), (3680, 155))

In [48]:
data = pd.concat([train, test]).reset_index(drop=True)


In [49]:
data

,child_id,data_year,child_date,child_age,child_grant,child_years_in_programme,child_height,child_observe_attentive,child_observe_concentrated,child_observe_diligent,child_observe_interested,child_observe_total,child_gender,child_dob,child_zha,child_stunted,child_attends,child_attendance,child_age_group,id_mn_best,prov_best,id_dc_best,dc_best,mn_best,ward_best,id_enumerator,id_facility,pra_free_play,pra_free_play_outdoor,pra_groupings,pra_groupings_1,pra_groupings_2,pra_groupings_3,pra_groupings_4,pra_groupings_5,pra_engaged,pra_agency_choice,pra_agency_explore,pra_agency_questions,pra_agency_understand,pra_agency_play,pra_agency_learn,pra_agency_order,pra_salary,pra_ind,pri_mobile,pri_school,pri_holidays,pri_days,pri_time_open_hours,pri_time_open_minutes,pri_time_close_hours,pri_time_close_minutes,pri_separate,pri_language,pri_language_1,pri_language_2,pri_language_3,pri_language_4,pri_language_5,pri_language_6,pri_language_7,pri_language_8,pri_language_9,pri_language_10,pri_language_11,pri_toys,pri_aftercare,pri_fees,pri_fees_amount,pri_facilities,pri_land,pri_bank,pri_transport,pri_subsidy,pri_funding_6,pri_funding_donations,pri_funding_97,pri_fees_amount_pv,teacher_social_peers,teacher_social_nonaggressive,teacher_social_cooperate,teacher_social_assistance,teacher_social_ideas,teacher_social_initiative,teacher_emotional_understand,teacher_emotional_appropriate,teacher_emotional_independent,teacher_emotional_adjust,teacher_emotional_confidence,teacher_emotional_selfstarter,teacher_social_total,teacher_selfcare_total,teacher_social_met,teacher_emotional_total,teacher_emotional_met,teacher_selfcare_met,hle_ind,obs_area,obs_water,obs_handwashing,obs_toilet,obs_toilet_1,obs_toilet_4,obs_toilet_0,obs_toilets_gender,obs_building,obs_shared,obs_outdoor,obs_fence,obs_gate,obs_access,id_ward,id_prov,count_register_all,count_staff_all,count_children_present,count_children_precovid,latitude,longitude,count_register_gender,count_register_race_african,count_register_race_coloured,count_register_race_white,count_register_race,count_staff_salary_paid,count_staff_salary_unpaid,count_staff_salary,count_staff_gender_female,count_staff_gender_male,count_staff_gender_other,count_staff_gender,count_staff_contract,count_staff_time_full,count_staff_time,count_staff_qual_skills,certificate_register,census,urban,quintile,language_child,language_assessment,sef_ind,language_match,elp_ind,gps_ind,pre_covid,ses_proxy,quintile_used,id_facility_n,id_ward_n,id_mn_n,id_dc_n,id_prov_n,ses_cat,target
0,ID_SYSJ2FM0D,2022.0,2022-02-03,59.000000,NaN,NaN,NaN,Sometimes,Sometimes,Sometimes,Sometimes,4.0,Female,2017-02-06,NaN,NaN,NaN,NaN,50-59 months,GT421,GAUTENG,DC42,SEDIBENG,EMFULENI,39.0,20005.0,761.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Does not meet standard,NaN,Does not meet standard,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74201039.0,GT,38.0,NaN,NaN,NaN,-26.551048,27.879824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,2.0,Sesotho,Sesotho,Yes,1.0,No,Yes,Post COVID,2.0,Yes,7.0,14.0,107.0,134.0,1051.0,NaN,51.500000
1,ID_J5BTFOZR3,2019.0,NaN,60.163933,NaN,1st year in the programme,103.000000,Sometimes,Almost never,Sometimes,Often,4.0,Female,NaN,-1.356791,Normal,NaN,NaN,60-69 months,NaN,KWAZULU-NATAL,DC22,UMGUNGUNDLOVU,NaN,NaN,NaN,458.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,4.0,Does not meet standard,7.0,Does not meet standard,Meets the standard,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## Feature Engineering Based of EDA 

In [50]:
data['child_dob'] = pd.to_datetime(data['child_dob'])
data['child_date'] = pd.to_datetime(data['child_date'])
data['diffday'] = (data['child_date'] - data['child_dob']).dt.days
data['diffdayage']=(data['diffday']-data["child_age"])/data["child_age"]
data["hage"]=data["child_age"]*data["child_height"]
data["total"]=data["child_observe_total"]+data["teacher_social_total"]+data["teacher_emotional_total"]
data.loc[data["pri_fees_amount"] > 750, "child_grant"] = "No"
data.loc[data["language_match"] == 1.0, "child_grant"] = "Yes"
data.loc[data["pra_salary"] == "More than R6000", "child_grant"] = "No"
data.loc[data["pra_salary"] == "R 5000 – R5999", "child_grant"] = "No"
data.loc[data["pra_salary"] == "R0", "child_grant"] = "Yes"
data.loc[data["pra_salary"] == "Less than R500 per month", "child_grant"] = "Yes"
data.loc[data["pri_fees"] == "No", "child_grant"] = "Yes"
data.loc[data["language_child"] == "English", "child_grant"] = "No"
data.loc[data["teacher_social_total"] == 24, "child_grant"] = "Yes"
data.loc[data["teacher_social_total"] == 0, "child_grant"] = "No"
data['child_attends'] = data['child_attends'].fillna('No')
data['child_observe_total']=data["child_observe_total"].fillna(0)

class_scores = {'Almost never': 0, 'Sometimes': 1, 'Often': 2,'Almost always':3}

# create a function to calculate the total score for each row
def calculate_total_score(row):
    total_score = 0
    for col in ['child_observe_attentive', 'child_observe_concentrated', 'child_observe_diligent','child_observe_interested']:
        class_score = class_scores.get(row[col], 0)
        total_score += class_score
    return total_score

# apply the function to the three columns and assign the result to a new column
data['child_observe_total'] = data.apply(calculate_total_score, axis=1)










## Filling Numerical features with Mean and Categorical features with the most frequent 

In [51]:
data.shape
num_cols = data.select_dtypes(include='number').columns
# fill NaN values in numerical columns with the mean value of the column
data[num_cols] = data[num_cols].fillna(data[num_cols].mean())

data = data.fillna(data.mode().iloc[0])
data=data.fillna(0)

## Transforming Ids from numbers to categories and applying mean encoder

In [52]:
data['two'] = data[['id_enumerator', 'id_facility']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

data["id_enumerator"] = data["id_enumerator"].astype(str)
data["id_facility"] = data["id_facility"].astype(str)
data["id_ward"] = data["id_ward"].astype(str)


data["ward_best"] = data["ward_best"].astype(str)




import category_encoders as ce

def median_encoder(x, y):
    med = y.median()
    return x.map(y.groupby(x).median().get)  # replace mean with median

# Define custom function to use standard deviation instead of mean
def std_encoder(x, y):
    std = y.std()
    return x.map(y.groupby(x).std().get)  # replace mean with std

# Encode using median instead of mean

# encode id_facility column using target encoding
encoder = ce.TargetEncoder()
data['child_years_in_programme1'] = encoder.fit_transform(data['child_years_in_programme'], data['child_zha'])
data['child_stunted2'] = encoder.fit_transform(data['child_stunted'], data['child_observe_total'])

data['faza'] = encoder.fit_transform(data['id_facility'], data['child_observe_total'])
data['faza2'] = encoder.fit_transform(data['id_facility'], data['child_age'])
data['faza1'] = encoder.fit_transform(data['id_facility'], data['child_height'])
data['faza3'] = encoder.fit_transform(data['id_facility'], data['child_zha'])
data['faza4'] = encoder.fit_transform(data['id_facility'], data['pri_fees_amount'])
data['faza5'] = encoder.fit_transform(data['id_facility'], data['pri_fees_amount_pv'])
data['faza6'] = encoder.fit_transform(data['id_facility'], data['teacher_social_total'])
data['faza7'] = encoder.fit_transform(data['id_facility'], data['teacher_selfcare_total'])
data['faza8'] = encoder.fit_transform(data['id_facility'], data['teacher_emotional_total'])
data['faza9'] = encoder.fit_transform(data['id_facility'], data['count_register_all'])
data['faza10'] = encoder.fit_transform(data['id_facility'], data['count_children_present'])
data['faza11'] = encoder.fit_transform(data['id_facility'], data['count_children_precovid'])
data['faza14'] = encoder.fit_transform(data['id_facility'], data['child_years_in_programme1'])
data['faza15'] = encoder.fit_transform(data['id_facility'], data['hage'])
data['faza16'] = encoder.fit_transform(data['id_facility'], data['total'])
data['faza17'] = encoder.fit_transform(data['id_facility'], data['count_register_race_coloured'])
data['faza18'] = encoder.fit_transform(data['id_facility'], data['count_register_race_white'])
data['faza19'] = encoder.fit_transform(data['id_facility'], data['count_register_race'])
data['faza20'] = encoder.fit_transform(data['id_facility'], data['count_staff_salary'])
data['faza21'] = encoder.fit_transform(data['id_facility'], data['count_staff_qual_skills'])





data['fazaa'] = encoder.fit_transform(data['id_enumerator'], data['child_observe_total'])
data['faza2a'] = encoder.fit_transform(data['id_enumerator'], data['child_age'])
data['faza1a'] = encoder.fit_transform(data['id_enumerator'], data['child_height'])
data['faza3a'] = encoder.fit_transform(data['id_enumerator'], data['child_zha'])
data['faza4a'] = encoder.fit_transform(data['id_enumerator'], data['pri_fees_amount'])
data['faza5a'] = encoder.fit_transform(data['id_enumerator'], data['pri_fees_amount_pv'])
data['faza6a'] = encoder.fit_transform(data['id_enumerator'], data['teacher_social_total'])
data['faza7a'] = encoder.fit_transform(data['id_enumerator'], data['teacher_selfcare_total'])
data['faza8a'] = encoder.fit_transform(data['id_enumerator'], data['teacher_emotional_total'])
data['fazvbcgba9'] = encoder.fit_transform(data['id_enumerator'], data['count_register_all'])
data['fasdvza10'] = encoder.fit_transform(data['id_enumerator'], data['count_children_present'])
data['fadwfvza11'] = encoder.fit_transform(data['id_enumerator'], data['count_children_precovid'])
data['fazaxvf144'] = encoder.fit_transform(data['id_enumerator'], data['child_years_in_programme1'])
data['faza15sdjns'] = encoder.fit_transform(data['id_enumerator'], data['hage'])
data['faza15xcknxc '] = encoder.fit_transform(data['id_enumerator'], data['total'])
data['faza17dqsfvfv'] = encoder.fit_transform(data['id_enumerator'], data['count_register_race_coloured'])
data['faza18wdvc'] = encoder.fit_transform(data['id_enumerator'], data['count_register_race_white'])
data['faza19fvdfvqd'] = encoder.fit_transform(data['id_enumerator'], data['count_register_race'])
data['faza20xvwdfv'] = encoder.fit_transform(data['id_enumerator'], data['count_staff_salary'])
data['faza21jlk'] = encoder.fit_transform(data['id_enumerator'], data['count_staff_qual_skills'])








## another costum encoder to work with sum,median,std

In [53]:
import numpy as np

class CustomEncoder:
    def __init__(self, func=np.mean):
        self.func = func
    
    def transform(self, groupby_col, target_col, data):
        grouped = data.groupby(groupby_col)[target_col]
        if self.func == np.mean:
            result = grouped.mean()
        elif self.func == np.median:
            result = grouped.median()
        elif self.func == np.sum:
            result = grouped.sum()
        elif self.func == np.std:
            result = grouped.std()
        elif self.func == np.min:
            result = grouped.min()
        elif self.func == np.max:
            result = grouped.max()
    
        else:
            raise ValueError("Invalid function")
        return data[groupby_col].map(result)


In [54]:
encoder = CustomEncoder(func=np.std)
data['fazastd1'] = encoder.transform('id_facility', 'child_observe_total', data)
data['fazastd2'] = encoder.transform('id_facility', 'child_age', data)
data['fazastd3'] = encoder.transform('id_facility', 'child_height', data)
data['fazastd4'] = encoder.transform('id_facility', 'child_zha', data)
data['fazastd5'] = encoder.transform('id_facility', 'pri_fees_amount', data)
data['fazastd6'] = encoder.transform('id_facility', 'pri_fees_amount_pv', data)
data['fazastd7'] = encoder.transform('id_facility', 'teacher_social_total', data)
data['fazastd8'] = encoder.transform('id_facility', 'teacher_selfcare_total', data)
data['fazastd9'] = encoder.transform('id_facility', 'teacher_emotional_total', data)
data['fazastd10'] = encoder.transform('id_facility', 'count_register_all', data)
data['fazastd11'] = encoder.transform('id_facility', 'count_children_present', data)
data['fazastd12'] = encoder.transform('id_facility', 'count_children_precovid', data)
data['fazastd13'] = encoder.transform('id_facility', 'child_years_in_programme1', data)
data['fazastd14'] = encoder.transform('id_facility', 'hage', data)
data['fazastd15'] = encoder.transform('id_facility', 'total', data)
data['fazastd16'] = encoder.transform('id_facility', 'count_register_race_coloured', data)
data['fazastd17'] = encoder.transform('id_facility', 'count_register_race_white', data)
data['fazastd18'] = encoder.transform('id_facility', 'count_register_race', data)
data['fazastd19'] = encoder.transform('id_facility', 'count_staff_salary', data)
data['fazastd20'] = encoder.transform('id_facility', 'count_staff_qual_skills', data)


data['fazaastd1'] = encoder.transform('id_enumerator', 'child_observe_total', data)
data['fazaastd2'] = encoder.transform('id_enumerator', 'child_age', data)
data['fazaastd3'] = encoder.transform('id_enumerator', 'child_height', data)
data['fazaastd4'] = encoder.transform('id_enumerator', 'child_zha', data)
data['fazaastd5'] = encoder.transform('id_enumerator', 'pri_fees_amount', data)
data['fazaastd6'] = encoder.transform('id_enumerator', 'pri_fees_amount_pv', data)
data['fazaastd7'] = encoder.transform('id_enumerator', 'teacher_social_total', data)
data['fazaastd8'] = encoder.transform('id_enumerator', 'teacher_selfcare_total', data)
data['fazaastd9'] = encoder.transform('id_enumerator', 'teacher_emotional_total', data)
data['fazaastd10'] = encoder.transform('id_enumerator', 'count_register_all', data)
data['fazaastd11'] = encoder.transform('id_enumerator', 'count_children_present', data)
data['fazaastd12'] = encoder.transform('id_enumerator', 'count_children_precovid', data)
data['fazaastd13'] = encoder.transform('id_enumerator', 'child_years_in_programme1', data)
data['fazaastd14'] = encoder.transform('id_enumerator', 'hage', data)
data['fazaastd15'] = encoder.transform('id_enumerator', 'total', data)
data['fazaastd16'] = encoder.transform('id_enumerator', 'count_register_race_coloured', data)
data['fazaastd17'] = encoder.transform('id_enumerator', 'count_register_race_white', data)
data['fazaastd18'] = encoder.transform('id_enumerator', 'count_register_race', data)
data['fazaastd19'] = encoder.transform('id_enumerator', 'count_staff_salary', data)
data['fazaastd20'] = encoder.transform('id_enumerator', 'count_staff_qual_skills', data)


encoder = CustomEncoder(func=np.sum)
data['fazastd11'] = encoder.transform('id_facility', 'child_observe_total', data)
data['fazastd21'] = encoder.transform('id_facility', 'child_age', data)
data['fazastd31'] = encoder.transform('id_facility', 'child_height', data)
data['fazastd41'] = encoder.transform('id_facility', 'child_zha', data)
data['fazastd51'] = encoder.transform('id_facility', 'pri_fees_amount', data)
data['fazastd61'] = encoder.transform('id_facility', 'pri_fees_amount_pv', data)
data['fazastd71'] = encoder.transform('id_facility', 'teacher_social_total', data)
data['fazastd81'] = encoder.transform('id_facility', 'teacher_selfcare_total', data)
data['fazastd91'] = encoder.transform('id_facility', 'teacher_emotional_total', data)
data['fazastd101'] = encoder.transform('id_facility', 'count_register_all', data)
data['fazastd111'] = encoder.transform('id_facility', 'count_children_present', data)
data['fazastd121'] = encoder.transform('id_facility', 'count_children_precovid', data)
data['fazastd131'] = encoder.transform('id_facility', 'child_years_in_programme1', data)
data['fazastd141'] = encoder.transform('id_facility', 'hage', data)
data['fazastd151'] = encoder.transform('id_facility', 'total', data)
data['fazastd161'] = encoder.transform('id_facility', 'count_register_race_coloured', data)
data['fazastd171'] = encoder.transform('id_facility', 'count_register_race_white', data)
data['fazastd181'] = encoder.transform('id_facility', 'count_register_race', data)
data['fazastd191'] = encoder.transform('id_facility', 'count_staff_salary', data)
data['fazastd201'] = encoder.transform('id_facility', 'count_staff_qual_skills', data)


data['fazaastd11'] = encoder.transform('id_enumerator', 'child_observe_total', data)
data['fazaastd21'] = encoder.transform('id_enumerator', 'child_age', data)
data['fazaastd31'] = encoder.transform('id_enumerator', 'child_height', data)
data['fazaastd41'] = encoder.transform('id_enumerator', 'child_zha', data)
data['fazaastd51'] = encoder.transform('id_enumerator', 'pri_fees_amount', data)
data['fazaastd61'] = encoder.transform('id_enumerator', 'pri_fees_amount_pv', data)
data['fazaastd71'] = encoder.transform('id_enumerator', 'teacher_social_total', data)
data['fazaastd81'] = encoder.transform('id_enumerator', 'teacher_selfcare_total', data)
data['fazaastd91'] = encoder.transform('id_enumerator', 'teacher_emotional_total', data)
data['fazaastd110'] = encoder.transform('id_enumerator', 'count_register_all', data)
data['fazaastd111'] = encoder.transform('id_enumerator', 'count_children_present', data)
data['fazaastd121'] = encoder.transform('id_enumerator', 'count_children_precovid', data)
data['fazaastd131'] = encoder.transform('id_enumerator', 'child_years_in_programme1', data)
data['fazaastd141'] = encoder.transform('id_enumerator', 'hage', data)
data['fazaastd151'] = encoder.transform('id_enumerator', 'total', data)
data['fazaastd161'] = encoder.transform('id_enumerator', 'count_register_race_coloured', data)
data['fazaastd171'] = encoder.transform('id_enumerator', 'count_register_race_white', data)
data['fazaastd181'] = encoder.transform('id_enumerator', 'count_register_race', data)
data['fazaastd191'] = encoder.transform('id_enumerator', 'count_staff_salary', data)
data['fazaastd201'] = encoder.transform('id_enumerator', 'count_staff_qual_skills', data)


encoder = CustomEncoder(func=np.median)
data['fazastd111'] = encoder.transform('id_facility', 'child_observe_total', data)
data['fazastd211'] = encoder.transform('id_facility', 'child_age', data)
data['fazastd311'] = encoder.transform('id_facility', 'child_height', data)
data['fazastd411'] = encoder.transform('id_facility', 'child_zha', data)
data['fazastd511'] = encoder.transform('id_facility', 'pri_fees_amount', data)
data['fazastd611'] = encoder.transform('id_facility', 'pri_fees_amount_pv', data)
data['fazastd711'] = encoder.transform('id_facility', 'teacher_social_total', data)
data['fazastd811'] = encoder.transform('id_facility', 'teacher_selfcare_total', data)
data['fazastd911'] = encoder.transform('id_facility', 'teacher_emotional_total', data)
data['fazastd1011'] = encoder.transform('id_facility', 'count_register_all', data)
data['fazastd1111'] = encoder.transform('id_facility', 'count_children_present', data)
data['fazastd1211'] = encoder.transform('id_facility', 'count_children_precovid', data)
data['fazastd1311'] = encoder.transform('id_facility', 'child_years_in_programme1', data)
data['fazastd1411'] = encoder.transform('id_facility', 'hage', data)
data['fazastd1511'] = encoder.transform('id_facility', 'total', data)
data['fazastd1611'] = encoder.transform('id_facility', 'count_register_race_coloured', data)
data['fazastd1711'] = encoder.transform('id_facility', 'count_register_race_white', data)
data['fazastd1811'] = encoder.transform('id_facility', 'count_register_race', data)
data['fazastd1911'] = encoder.transform('id_facility', 'count_staff_salary', data)
data['fazastd2011'] = encoder.transform('id_facility', 'count_staff_qual_skills', data)


data['fazaastd111'] = encoder.transform('id_enumerator', 'child_observe_total', data)
data['fazaastd211'] = encoder.transform('id_enumerator', 'child_age', data)
data['fazaastd311'] = encoder.transform('id_enumerator', 'child_height', data)
data['fazaastd411'] = encoder.transform('id_enumerator', 'child_zha', data)
data['fazaastd511'] = encoder.transform('id_enumerator', 'pri_fees_amount', data)
data['fazaastd611'] = encoder.transform('id_enumerator', 'pri_fees_amount_pv', data)
data['fazaastd711'] = encoder.transform('id_enumerator', 'teacher_social_total', data)
data['fazaastd811'] = encoder.transform('id_enumerator', 'teacher_selfcare_total', data)
data['fazaastd911'] = encoder.transform('id_enumerator', 'teacher_emotional_total', data)
data['fazaastd1101'] = encoder.transform('id_enumerator', 'count_register_all', data)
data['fazaastd1111'] = encoder.transform('id_enumerator', 'count_children_present', data)
data['fazaastd1211'] = encoder.transform('id_enumerator', 'count_children_precovid', data)
data['fazaastd1311'] = encoder.transform('id_enumerator', 'child_years_in_programme1', data)
data['fazaastd1411'] = encoder.transform('id_enumerator', 'hage', data)
data['fazaastd1511'] = encoder.transform('id_enumerator', 'total', data)
data['fazaastd1611'] = encoder.transform('id_enumerator', 'count_register_race_coloured', data)
data['fazaastd1711'] = encoder.transform('id_enumerator', 'count_register_race_white', data)
data['fazaastd1811'] = encoder.transform('id_enumerator', 'count_register_race', data)
data['fazaastd1911'] = encoder.transform('id_enumerator', 'count_staff_salary', data)
data['fazaastd2011'] = encoder.transform('id_enumerator', 'count_staff_qual_skills', data)



## OneHotEncoding for categorical variables less than 500 class

In [55]:
from sklearn.preprocessing import OneHotEncoder
df=data.copy()
# create a sample dataframe with categorical variables
# loop over each categorical column in the dataframe
for col in df.select_dtypes(include=['object']):
    # check if the column has less than 150 unique values
    if len(df[col].unique()) < 500:
        
        encoder = OneHotEncoder(categories='auto', drop='first')
        one_hot = encoder.fit_transform(data[[col]])
        
        # create new column names with the prefix
        prefix = col + '_'
        new_cols = [prefix + str(i) for i in range(one_hot.shape[1])]
        
        # append the encoded columns to the original dataframe
        df_one_hot = pd.DataFrame(one_hot.toarray(), columns=new_cols)
        data = pd.concat([data, df_one_hot], axis=1)
        
        # drop the original column from the dataframe
        #data = data.drop(col, axis=1)


## dropping categorical columns

In [56]:
cat_cols = data.select_dtypes(include=['object', 'category']).columns

# drop categorical columns
data = data.drop(cat_cols, axis=1)


## StanderScaler

In [57]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

target_column = 'target'

# identify the numerical columns
numerical_columns = data.select_dtypes(include=["number"]).columns

# remove the target column from the list of numerical columns
numerical_columns = numerical_columns.drop(target_column)

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to the numerical columns
scaler.fit(data[numerical_columns])

# transform the numerical columns using the scaler
scaled_features = scaler.transform(data[numerical_columns])

# convert the scaled features array to a Pandas dataframe
scaled_features_df = pd.DataFrame(scaled_features, columns=numerical_columns)

# concatenate the target column with the scaled features dataframe
data = pd.concat([data[target_column], data[numerical_columns]], axis=1)


In [58]:
train = data[:train.shape[0]].reset_index(drop=True)
test = data[train.shape[0]:].reset_index(drop=True)


In [59]:
pd.set_option('display.max_columns', None)
numerical_columns = data.select_dtypes(include=["number"]).columns

l = numerical_columns.drop("target")


In [60]:
l=l.to_list()


In [61]:

y = train.target

X = train[l]


## Testing with LinearRegression after choosing best random_estate 254

In [62]:

X=X.fillna(0)
# Split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state =254)

# Train a model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Check score
mean_squared_error(y_test, y_pred, squared=False)

9.042782793338471

In [63]:
model = LinearRegression()
model.fit(X, y)

# Make predictions
y_pred = model.predict(X_test)

# Check score
mean_squared_error(y_test, y_pred, squared=False)

7.851470463968846

## BayesianRidge Regressor

In [64]:
from sklearn.linear_model import BayesianRidge

paramlasso={'alpha_1': 9.8835984973077e-07, 'alpha_2': 3.8198792835159964e-08, 'lambda_1': 1.0163492663828834e-10, 'lambda_2': 8.414116938780497e-07}
bayes = BayesianRidge(**paramlasso)

bayes.fit(X, y)



# Predict on test set and calculate RMSE score
y_pred = bayes.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Best hyperparameters:', best_params)
print('Best RMSE score:', rmse)


Best hyperparameters: {'learning_rate': 0.03073647294595579, 'num_leaves': 52, 'max_depth': 20, 'min_child_samples': 7, 'min_child_weight': 0.06295677325129843, 'subsample': 0.9688192953085165, 'colsample_bytree': 0.80144841136625, 'reg_alpha': 6.661629258332502, 'reg_lambda': 1.228538673958893, 'n_estimators': 638, 'min_split_gain': 0.2774307584715542, 'max_bin': 450, 'subsample_freq': 1, 'bagging_fraction': 0.9489397998472825, 'bagging_freq': 7, 'feature_fraction': 0.5203836918415207}
Best RMSE score: 8.694876683648754


## LGBM Hyperparamater Tuning with Bayesian optimization  (Optuna)

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1234)

def objective(trial):
    # Define hyperparameters to be tuned
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 5, 10000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 500),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.01, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'max_bin': trial.suggest_int('max_bin', 100, 500),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'random_state': 1234,
    }    
    # Train LightGBM model with current set of hyperparameters
    lgbm = lgb.LGBMRegressor(**params)
    lgbm.fit(X_train, y_train, verbose=False)
    
    # Predict on test set and calculate RMSE score
    y_pred = lgbm.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

# Set up the Optuna study and run trials
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

# Train LightGBM model with best set of hyperparameters found by Optuna
best_params = study.best_params
best_lgbm = lgb.LGBMRegressor(**best_params)
best_lgbm.fit(X_train, y_train, verbose=False)

# Predict on test set and calculate RMSE score
y_pred = best_lgbm.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Best hyperparameters:', best_params)
print('Best RMSE score:', rmse)


[I 2023-04-25 22:22:42,791] A new study created in memory with name: no-name-b2a2b0dc-6d48-4881-8b9e-b73d2297b1a0
[I 2023-04-25 22:23:23,158] Trial 0 finished with value: 9.41964392941785 and parameters: {'learning_rate': 0.059068417349712556, 'num_leaves': 986, 'max_depth': 6826, 'min_child_samples': 244, 'min_child_weight': 0.049205062334244834, 'subsample': 0.5481617303791957, 'colsample_bytree': 0.7737821230844639, 'reg_alpha': 2.5670780248487577, 'reg_lambda': 0.16600757037863761, 'n_estimators': 2690, 'min_split_gain': 0.6124168409263764, 'max_bin': 234, 'subsample_freq': 7, 'bagging_fraction': 0.9267640156915613, 'bagging_freq': 3, 'feature_fraction': 0.8575868133718844}. Best is trial 0 with value: 9.41964392941785.


[LightGBM] [Warning] bagging_fraction is set=0.7219162353232573, subsample=0.6682267055526363 will be ignored. Current value: bagging_fraction=0.7219162353232573
[LightGBM] [Warning] feature_fraction is set=0.7429897149913156, colsample_bytree=0.8712620321850484 will be ignored. Current value: feature_fraction=0.7429897149913156
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=9 will be ignored. Current value: bagging_freq=6


[I 2023-04-25 22:23:59,310] Trial 1 finished with value: 8.96846200911522 and parameters: {'learning_rate': 0.01861719066700852, 'num_leaves': 551, 'max_depth': 5702, 'min_child_samples': 286, 'min_child_weight': 0.041766733891377084, 'subsample': 0.6682267055526363, 'colsample_bytree': 0.8712620321850484, 'reg_alpha': 8.246601811894498, 'reg_lambda': 0.031064617319553556, 'n_estimators': 3740, 'min_split_gain': 0.961163954400216, 'max_bin': 322, 'subsample_freq': 9, 'bagging_fraction': 0.7219162353232573, 'bagging_freq': 6, 'feature_fraction': 0.7429897149913156}. Best is trial 1 with value: 8.96846200911522.


[LightGBM] [Warning] bagging_fraction is set=0.8247945298683639, subsample=0.5588387043150709 will be ignored. Current value: bagging_fraction=0.8247945298683639
[LightGBM] [Warning] feature_fraction is set=0.8752690386520188, colsample_bytree=0.6510123798117772 will be ignored. Current value: feature_fraction=0.8752690386520188
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=10 will be ignored. Current value: bagging_freq=5


[I 2023-04-25 22:24:31,861] Trial 2 finished with value: 9.614136446931925 and parameters: {'learning_rate': 0.001402741586079131, 'num_leaves': 270, 'max_depth': 232, 'min_child_samples': 409, 'min_child_weight': 0.02613813172703264, 'subsample': 0.5588387043150709, 'colsample_bytree': 0.6510123798117772, 'reg_alpha': 0.36076381679022435, 'reg_lambda': 9.605112057168501, 'n_estimators': 3649, 'min_split_gain': 0.35160801313739487, 'max_bin': 169, 'subsample_freq': 10, 'bagging_fraction': 0.8247945298683639, 'bagging_freq': 5, 'feature_fraction': 0.8752690386520188}. Best is trial 1 with value: 8.96846200911522.


...

[LightGBM] [Warning] bagging_fraction is set=0.6873364672649793, subsample=0.8128853467944687 will be ignored. Current value: bagging_fraction=0.6873364672649793
[LightGBM] [Warning] feature_fraction is set=0.852632920452934, colsample_bytree=0.5484474211162603 will be ignored. Current value: feature_fraction=0.852632920452934
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=2 will be ignored. Current value: bagging_freq=5


[I 2023-04-25 23:33:46,144] Trial 101 finished with value: 8.934159497210612 and parameters: {'learning_rate': 0.007183932148976933, 'num_leaves': 780, 'max_depth': 6022, 'min_child_samples': 259, 'min_child_weight': 0.04030904517217823, 'subsample': 0.8128853467944687, 'colsample_bytree': 0.5484474211162603, 'reg_alpha': 0.23399077361172319, 'reg_lambda': 0.015044192745011169, 'n_estimators': 3517, 'min_split_gain': 0.5421216479943575, 'max_bin': 162, 'subsample_freq': 2, 'bagging_fraction': 0.6873364672649793, 'bagging_freq': 5, 'feature_fraction': 0.852632920452934}. Best is trial 85 with value: 8.884114380041806.


[LightGBM] [Warning] bagging_fraction is set=0.7341329824769162, subsample=0.7800055496674556 will be ignored. Current value: bagging_fraction=0.7341329824769162
[LightGBM] [Warning] feature_fraction is set=0.8373548848794903, colsample_bytree=0.568128477493509 will be ignored. Current value: feature_fraction=0.8373548848794903
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=2 will be ignored. Current value: bagging_freq=5


[I 2023-04-25 23:34:18,532] Trial 102 finished with value: 8.903849494012732 and parameters: {'learning_rate': 0.008500243635610437, 'num_leaves': 814, 'max_depth': 7218, 'min_child_samples': 221, 'min_child_weight': 0.04338757006732821, 'subsample': 0.7800055496674556, 'colsample_bytree': 0.568128477493509, 'reg_alpha': 0.18352427009743585, 'reg_lambda': 0.012528378317118812, 'n_estimators': 2873, 'min_split_gain': 0.6023081362025283, 'max_bin': 121, 'subsample_freq': 2, 'bagging_fraction': 0.7341329824769162, 'bagging_freq': 5, 'feature_fraction': 0.8373548848794903}. Best is trial 85 with value: 8.884114380041806.


[LightGBM] [Warning] bagging_fraction is set=0.7355931272574537, subsample=0.7663925868806094 will be ignored. Current value: bagging_fraction=0.7355931272574537
[LightGBM] [Warning] feature_fraction is set=0.8376524828517931, colsample_bytree=0.5343568238838858 will be ignored. Current value: feature_fraction=0.8376524828517931
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=2 will be ignored. Current value: bagging_freq=5


## Testing LGBM after HP

In [64]:
import lightgbm as lgb

best_params= {'learning_rate': 0.03073647294595579, 'num_leaves': 52, 'max_depth': 20, 'min_child_samples': 7, 'min_child_weight': 0.06295677325129843, 'subsample': 0.9688192953085165, 'colsample_bytree': 0.80144841136625, 'reg_alpha': 6.661629258332502, 'reg_lambda': 1.228538673958893, 'n_estimators': 638, 'min_split_gain': 0.2774307584715542, 'max_bin': 450, 'subsample_freq': 1, 'bagging_fraction': 0.9489397998472825, 'bagging_freq': 7, 'feature_fraction': 0.5203836918415207}
best_lgbm = lgb.LGBMRegressor(**best_params)

best_lgbm.fit(X, y, verbose=False)



# Predict on test set and calculate RMSE score
y_pred = best_lgbm.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Best hyperparameters:', best_params)
print('Best RMSE score:', rmse)


[LightGBM] [Warning] feature_fraction is set=0.5203836918415207, colsample_bytree=0.80144841136625 will be ignored. Current value: feature_fraction=0.5203836918415207
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=1 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.9489397998472825, subsample=0.9688192953085165 will be ignored. Current value: bagging_fraction=0.9489397998472825
Best hyperparameters: {'learning_rate': 0.03073647294595579, 'num_leaves': 52, 'max_depth': 20, 'min_child_samples': 7, 'min_child_weight': 0.06295677325129843, 'subsample': 0.9688192953085165, 'colsample_bytree': 0.80144841136625, 'reg_alpha': 6.661629258332502, 'reg_lambda': 1.228538673958893, 'n_estimators': 638, 'min_split_gain': 0.2774307584715542, 'max_bin': 450, 'subsample_freq': 1, 'bagging_fraction': 0.9489397998472825, 'bagging_freq': 7, 'feature_fraction': 0.5203836918415207}
Best RMSE score: 4.422270107666723


## Cross Validation for LGBM

In [210]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Define the number of folds for cross-validation
n_folds = 5

# Define the hyperparameters to be tuned
params = {'learning_rate': 0.03073647294595579, 'num_leaves': 52, 'max_depth': 20, 'min_child_samples': 7, 'min_child_weight': 0.06295677325129843, 'subsample': 0.9688192953085165, 'colsample_bytree': 0.80144841136625, 'reg_alpha': 6.661629258332502, 'reg_lambda': 1.228538673958893, 'n_estimators': 638, 'min_split_gain': 0.2774307584715542, 'max_bin': 450, 'subsample_freq': 1, 'bagging_fraction': 0.9489397998472825, 'bagging_freq': 7, 'feature_fraction': 0.5203836918415207}

# Create the LightGBM model with the specified hyperparameters
model = lgb.LGBMRegressor(**params)

# Define the cross-validation strategy
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize an empty array to store the RMSE scores
rmse_scores = []

# Loop through each fold and fit the model on the training set, and evaluate it on the validation set
for train_idx, val_idx in kf.split(X_train):
    X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]

    X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]

    model.fit(X_train_fold, y_train_fold, verbose=False)
    y_pred_fold = model.predict(X_val_fold)
    rmse_fold = mean_squared_error(y_val_fold, y_pred_fold, squared=False)
    rmse_scores.append(rmse_fold)

# Calculate the mean RMSE score across all folds
mean_rmse = np.mean(rmse_scores)

# Print the best hyperparameters and the mean RMSE score
print('Best hyperparameters:', params)
print('Mean RMSE score:', mean_rmse)


[LightGBM] [Warning] feature_fraction is set=0.5203836918415207, colsample_bytree=0.80144841136625 will be ignored. Current value: feature_fraction=0.5203836918415207
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=1 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.9489397998472825, subsample=0.9688192953085165 will be ignored. Current value: bagging_fraction=0.9489397998472825
[LightGBM] [Warning] feature_fraction is set=0.5203836918415207, colsample_bytree=0.80144841136625 will be ignored. Current value: feature_fraction=0.5203836918415207
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=1 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.9489397998472825, subsample=0.9688192953085165 will be ignored. Current value: bagging_fraction=0.9489397998472825
[LightGBM] [Warning] feature_fraction is set=0.5203836918415207, colsample_bytree=0.80144841136625 will be ignored. Current va

## Feature Importances for LGBM

In [154]:
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': best_lgbm.feature_importances_})
feature_importances = feature_importances.sort_values('importance', ascending=False).reset_index(drop=True)

# Print feature importances
print(feature_importances.head(50))


                    feature  importance
0                 child_zha        1101
1                 child_age         863
2                      hage         862
3              child_height         856
4                diffdayage         838
5                   diffday         754
6       child_observe_total         673
7                     total         576
8   teacher_emotional_total         485
9                  fazastd2         472
10                     faza         460
11                 fazastd1         441
12                    fazaa         363
13     teacher_social_total         359
14               fazastd211         357
15                 fazastd7         327
16                fazastd14         319
17                fazastd41         309
18                   faza14         307
19                    faza2         304
20                    faza8         303
21                 fazastd3         281
22               fazastd411         277
23                    faza3         275


## Same Work done for CatBoost Regressor 

In [47]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold

best_params={'learning_rate': 0.06171079611429389, 'depth': 6, 'l2_leaf_reg': 4.239644716420992, 'random_strength': 7.486454267792664, 'bagging_temperature': 0.18698588897880694, 'border_count': 166, 'iterations': 943, 'early_stopping_rounds': 16}

# Perform K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1234)
scores = []
for train_index, test_index in kf.split(X):
    # Split data into train and test sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create CatBoost data pool
    train_pool = Pool(X_train, y_train)
    test_pool = Pool(X_test, y_test)

    # Train the CatBoost model
    catboost = CatBoostRegressor(**best_params)
    catboost.fit(train_pool, eval_set=test_pool)

    # Evaluate the model on the test set
    score = catboost.get_best_score()['validation']['RMSE']
    scores.append(score)

# Print the average score across all folds
print('Average RMSE:', np.mean(scores))


0:	learn: 15.0754837	test: 14.4767625	best: 14.4767625 (0)	total: 5.77ms	remaining: 5.43s

1:	learn: 14.8944701	test: 14.3100080	best: 14.3100080 (1)	total: 11.2ms	remaining: 5.27s

2:	learn: 14.7665822	test: 14.1924408	best: 14.1924408 (2)	total: 16.2ms	remaining: 5.08s

3:	learn: 14.5643747	test: 13.9925006	best: 13.9925006 (3)	total: 20.8ms	remaining: 4.89s

4:	learn: 14.3573272	test: 13.7978647	best: 13.7978647 (4)	total: 25.3ms	remaining: 4.74s

5:	learn: 14.2487514	test: 13.6903060	best: 13.6903060 (5)	total: 29.6ms	remaining: 4.62s

6:	learn: 14.1145251	test: 13.5643708	best: 13.5643708 (6)	total: 34.2ms	remaining: 4.57s

7:	learn: 13.9365444	test: 13.3971467	best: 13.3971467 (7)	total: 38.5ms	remaining: 4.5s

8:	learn: 13.7661005	test: 13.2393369	best: 13.2393369 (8)	total: 43ms	remaining: 4.46s

9:	learn: 13.6567745	test: 13.1410427	best: 13.1410427 (9)	total: 47.3ms	remaining: 4.42s

10:	learn: 13.5581756	test: 13.0510211	best: 13.0510211 (10)	total: 51.5ms	remaining: 4.37s



In [ ]:
import optuna
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor

# define the objective function
def objective(trial):
    # Define hyperparameters to be optimized
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.3),
        'depth': trial.suggest_int('depth', 4, 16),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.01, 1000),
        'random_strength': trial.suggest_loguniform('random_strength', 0.01, 1000),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'iterations': trial.suggest_int('iterations', 50, 2000),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 5, 50),
        'logging_level': 'Silent'
    }
    
    # Fit CatBoostRegressor with current set of hyperparameters
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_test, y_test))
    
    # Calculate and return mean squared error on test set
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

# Print best set of hyperparameters and corresponding mean squared error
best_params = study.best_params
best_mse = study.best_value
print(f"Best parameters: {best_params}")
print(f"Best MSE: {best_mse}")

# Train LightGBM model with best set of hyperparameters found by Optuna
best_params = study.best_params
best_cat = CatBoostRegressor(**best_params)
best_cat.fit(X_train, y_train, verbose=False)

# Predict on test set and calculate RMSE score
y_pred = best_cat.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Best hyperparameters:', best_params)
print('Best RMSE score:', rmse)


[I 2023-04-24 00:15:19,534] A new study created in memory with name: no-name-0ca4e766-b270-4e4f-94ef-6b050fa71255

[I 2023-04-24 00:15:22,739] Trial 0 finished with value: 8.73032090902785 and parameters: {'learning_rate': 0.13792666295493428, 'depth': 7, 'l2_leaf_reg': 0.5536342793087746, 'random_strength': 48.24008022629101, 'bagging_temperature': 0.0768962398461634, 'border_count': 162, 'min_data_in_leaf': 35, 'iterations': 667, 'early_stopping_rounds': 7}. Best is trial 0 with value: 8.73032090902785.

[I 2023-04-24 00:16:42,355] Trial 1 finished with value: 9.122911314950706 and parameters: {'learning_rate': 0.142797781445808, 'depth': 12, 'l2_leaf_reg': 0.033584689103867714, 'random_strength': 67.92480407402026, 'bagging_temperature': 0.01017650865073659, 'border_count': 249, 'min_data_in_leaf': 1, 'iterations': 903, 'early_stopping_rounds': 47}. Best is trial 0 with value: 8.73032090902785.

[I 2023-04-24 00:16:53,214] Trial 2 finished with value: 13.560932797692452 and paramete

In [65]:
from catboost import CatBoostRegressor

cat_best_params2={'learning_rate': 0.0332920169935953, 'depth': 8, 'l2_leaf_reg': 5.675211232327334, 'random_strength': 59.80073587683864, 'bagging_temperature': 0.32631368454042525, 'border_count': 64, 'min_data_in_leaf': 12, 'iterations': 1799, 'early_stopping_rounds': 40}
cat_best_params={'learning_rate': 0.06171079611429389, 'depth': 6, 'l2_leaf_reg': 4.239644716420992, 'random_strength': 7.486454267792664, 'bagging_temperature': 0.18698588897880694, 'border_count': 166, 'iterations': 943, 'early_stopping_rounds': 16}
cat_best_params4={'learning_rate': 0.004248820134537801, 'depth': 12, 'l2_leaf_reg': 1.3658546660798294, 'random_strength': 0.5848595589714214, 'bagging_temperature': 0.017555412353386955, 'border_count': 32, 'min_data_in_leaf': 5, 'iterations': 1586, 'early_stopping_rounds': 22}
cat_best_params3={'learning_rate': 0.08201139125274072, 'depth': 8, 'l2_leaf_reg': 37.04529630486893, 'random_strength': 13.38579886562616, 'bagging_temperature': 0.3882996370198654, 'border_count': 40, 'min_data_in_leaf': 48, 'iterations': 1912, 'early_stopping_rounds': 50}
best_cat = CatBoostRegressor(**cat_best_params2)
best_cat.fit(X, y, verbose=False)

# Predict on test set and calculate RMSE score
y_pred = best_cat.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Best RMSE score:', rmse)


Best RMSE score: 5.553252092537232


## Testing GradientBoostingRegressor

In [195]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_param={'learning_rate': 0.00687457636992766, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 16, 'n_estimators': 1834}
best_gbr = GradientBoostingRegressor(**gbr_param)
best_gbr.fit(X, y)

# Predict on test set and calculate RMSE score
y_pred = best_gbr.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Best RMSE score:', rmse)


Best RMSE score: 4.393808553331527


## Testing Stacking Method

In [712]:
pred1 = np.zeros(train.shape[0])
pred2 = np.zeros(train.shape[0])
pred3 = np.zeros(train.shape[0])
pred4 = np.zeros(train.shape[0])

test1 = np.zeros(test.shape[0])
test2 = np.zeros(test.shape[0])
test3 = np.zeros(test.shape[0])
test4 = np.zeros(test.shape[0])

kf = KFold(n_splits=10,random_state=254,shuffle=True)
n=0

for trn_idx, test_idx in kf.split(train[l],train['target']):
    print(f"fold: {n+1}")
    X_tr,X_val=train[l].iloc[trn_idx],train[l].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    
    
    model1 = CatBoostRegressor(**cat_best_params2)
    model1.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],verbose=False)
    pred1[test_idx] = model1.predict(X_val)
    
    rmse1 = mean_squared_error(y_val, model1.predict(X_val), squared=False)
    print(": model1 rmse = {}".format(rmse1))

    
    model4 = LinearRegression()
    model4.fit(X_tr,y_tr)
    pred4[test_idx] = model4.predict(X_val)
    rmse4 = mean_squared_error(y_val, model4.predict(X_val), squared=False)
    print(": model4 rmse = {}".format(rmse4))
    print(": average all models rmse = {}".format((rmse1+rmse2+rmse3+rmse4)/4))

    n+=1
model1 = CatBoostRegressor(**cat_best_params2)
model1.fit(X,y,verbose=False)
    
test1 = model1.predict(test[l]) 

model4 = BayesianRidge(**paramlasso)
model4.fit(X,y)
test4 = model4.predict(test[l])



fold: 1

: model1 rmse = 8.707454065998862

: model4 rmse = 9.015749462928195

: average all models rmse = 8.999997401509724

fold: 2

: model1 rmse = 8.981448259491103

: model4 rmse = 10.183945282962668

: average all models rmse = 9.360544904891404

fold: 3

: model1 rmse = 9.846476145649621

: model4 rmse = 10.45635006920815

: average all models rmse = 9.644903072992403

fold: 4

: model1 rmse = 9.222928380269655

: model4 rmse = 9.616192939912544

: average all models rmse = 9.27897684932351

fold: 5

: model1 rmse = 9.124989862477621

: model4 rmse = 9.858950055530354

: average all models rmse = 9.315181498779953

fold: 6

: model1 rmse = 9.19184315809204

: model4 rmse = 14.72699874686913

: average all models rmse = 10.548906995518252

fold: 7

: model1 rmse = 9.620924949049227

: model4 rmse = 65.36213741266378

: average all models rmse = 23.314962109706215

fold: 8

: model1 rmse = 9.682702384137132

: model4 rmse = 10.109772416744786

: average all models rmse = 9.5173152

In [713]:
stacked_predictions = np.column_stack((pred1,pred4))
stacked_test_predictions = np.column_stack((test1,test4))

In [720]:
l1_train = pd.DataFrame(data={
    "cat": pred1.tolist(),
    "bayes": pred4.tolist(),
    "target":train.target
    })
l1_test = pd.DataFrame(data={
    "cat": test1.tolist(),
    "bayes": test4.tolist()
    })

In [727]:
y = l1_train.target

X = l1_train.drop(columns=["target"])


In [728]:
X

,cat,bayes
0,47.650526,48.062234
1,49.224204,46.367939
2,55.102445,56.605125
3,51.871946,56.873397
4,56.175847,45.092499
...,...,...
8580,30.422582,32.622613
8581,45.252102,44.674384
8582,53.306808,33.037352
8583,53.811190,53.290468


In [367]:
kf = KFold(n_splits=5,random_state=1234,shuffle=True)
final_prediction = np.zeros(test.shape[0])
rmse=[]  # list contains rmse for each fold
n=0
par={'learning_rate': 0.003966927486781653, 'num_leaves': 201, 'max_depth': 14, 'min_child_samples': 119, 'min_child_weight': 0.018315364263271884, 'subsample': 0.5800716862396311, 'colsample_bytree': 0.5677501725711916, 'reg_alpha': 0.02145047189669832, 'reg_lambda': 7.665480268450087, 'n_estimators': 743, 'min_split_gain': 0.23373739398754664, 'max_bin': 100, 'subsample_freq': 6, 'bagging_fraction': 0.9662909377114273, 'bagging_freq': 8, 'feature_fraction': 0.7009555002612379}
for trn_idx, test_idx in kf.split(stacked_predictions,train['target']):
    X_tr,X_val=stacked_predictions[trn_idx],stacked_predictions[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    
    meta_model = lgb.LGBMRegressor(**par)
    meta_model.fit(X_tr,y_tr,verbose=False)
    
    final_prediction +=meta_model.predict(stacked_test_predictions)/kf.n_splits
    rmse.append(mean_squared_error(y_val, meta_model.predict(X_val), squared=False))
    print(f"fold: {n+1}, rmse: {rmse[n]}")
    n+=1

fold: 1, rmse: 9.19614600383729

[LightGBM] [Warning] feature_fraction is set=0.7009555002612379, colsample_bytree=0.5677501725711916 will be ignored. Current value: feature_fraction=0.7009555002612379

[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=6 will be ignored. Current value: bagging_freq=8

[LightGBM] [Warning] bagging_fraction is set=0.9662909377114273, subsample=0.5800716862396311 will be ignored. Current value: bagging_fraction=0.9662909377114273

fold: 2, rmse: 9.334849024882361

[LightGBM] [Warning] feature_fraction is set=0.7009555002612379, colsample_bytree=0.5677501725711916 will be ignored. Current value: feature_fraction=0.7009555002612379

[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=6 will be ignored. Current value: bagging_freq=8

[LightGBM] [Warning] bagging_fraction is set=0.9662909377114273, subsample=0.5800716862396311 will be ignored. Current value: bagging_fraction=0.9662909377114273

fold: 3, rmse: 9.49494512369766

[LightGBM] [Warn

## RMSE Score for the Stacking method 

In [368]:
np.mean(rmse)

9.350808792209895

## Ensembling the scores of the two best boosting models ( Catboost+LGBM)

In [66]:
# Make predictions on the test 
set
preds = best_lgbm.predict(test[l].fillna(0))
catboostpreds = best_cat.predict(test[l].fillna(0))


In [1151]:
# Get the top five features for each predictions
from tqdm import tqdm

top_predictors = []

for i in tqdm(range(test.shape[0])):
  predictors = eli5.explain_prediction_df(estimator=model, doc=test[l].fillna(0).iloc[i], top = 15)
  top_predictors.append(predictors.feature.tolist())

100%|███████████████████████████████████████| 3680/3680 [01:04<00:00, 57.17it/s]


In [1153]:
# Submission file preparation
test = pd.read_csv('/home/fjaadari/Downloads/Test (4).csv')

predictors = pd.DataFrame(top_predictors, columns = ['feature_' + str(i) for i in range(1,16)])
predictors['target'] = preds
predictors['child_id'] = test.child_id
predictors = predictors[ss.columns.tolist()]
predictors.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,61.060146,data_year,pri_year,child_age,count_staff_salary,count_staff_gender_female,teacher_emotional_total,census_0,pri_calc_time_close_18,language_match,pri_days,pri_time_close_hours,count_staff_gender,count_staff_salary_paid,child_height,<BIAS>
1,ID_GQ6ONJ4FP,40.165440,data_year,pri_year,count_staff_gender_female,count_staff_salary,child_age,teacher_emotional_total,census_0,count_staff_salary_unpaid,pri_time_open_hours,pri_days,pri_time_close_hours,count_staff_gender,child_height,count_staff_salary_paid,<BIAS>
2,ID_YZ76CVRW3,45.254529,data_year,pri_year,child_age,count_staff_gender_female,obs_heating_census_5,teacher_emotional_total,count_staff_salary,census_0,language_match,count_staff_gender,pri_days,pri_time_close_hours,count_staff_salary_paid,child_height,<BIAS>
3,ID_BNINCRXH8,68.689604,data_year,count_staff_gender_female,count_staff_salary,pri_year,child_age,count_register_race_african,count_register_gender,count_staff_all,pri_days,count_register_race,pri_time_close_hours,child_height,count_staff_gender,count_staff_salary_paid,<BIAS>
4,ID_1U7GDTLRI,48.243100,data_year,pri_year,count_staff_gender_female,child_age,count_staff_salary,census_0,elp_ind_0,language_match,pri_time_open_hours,pri_days,pri_time_close_hours,count_staff_gender,count_staff_salary_paid,child_height,<BIAS>


## Getting the Final result after merging the two preds

In [1154]:
# Create csv to upload to Zindi for scoring
predictors.to_csv('subfinal.csv', index = False)

In [16]:
final_sub_boosting = pd.read_csv('/home/fjaadari/Downloads/subfinal.csv')

final_sub_boosting["target"]=preds*0.5+catboostpreds*0.5

In [ ]:
final_sub_boosting.to_csv('subfinalBoosting.csv', index = False)